In [ ]:
###get z-score transformed accessibility 
ls ../scD*.b37.calmd.cytosine.filtered.sort.GCH.strand.6plus2.bed | perl -ne 'chomp;$in=$_;$p=$in;$p=~s/\S+\/(scD_\w+\.\w+).b37.*.bed$/$1/;`perl /jet/home/dnaase/ssubexc.pl "perl /jet/home/dnaase/startup/software/bistools/Bis-tools/utils/bed6plus2bw_zscore.b37.pl $p $in" zscore_$p 4 2`'

##align z-score to FOS (biggest difference between two cell types).
ls scD*.methy_zscore.b37.bw | perl -ne 'chomp;$in=$_;$p=$in;$p=~s/\.methy_zscore.b37.bw$//;`perl /jet/home/dnaase/ssubexc.pl "perl /jet/home/dnaase/code/dnaase_google_code/bin/alignWigToBed.pl --alignment_mode 1 --average --prefix FOS.IMR90.distal.$p --locs /jet/home/dnaase/shared/data/tf_chip/IMR90/FOS.IMR90.ENCFF359IPV.optimalIDR.b37.distal.bed $in --data_matrix_scale 1200 --plot_x_axis_scale 1000 --plot_x_axis_step 500 --omit_plot_step" FOS_1kb_$p 2 1`'

##FOS (best performance, some red in blue)
library(Rtsne)
library(umap)
setwd("/jet/home/dnaase/startup/projects/hexa_seq/scNOMe_RNA_IMR90_GM_20220330/DNA/bam/acc")
mc<-read.table("/jet/home/dnaase/startup/projects/hexa_seq/scNOMe_RNA_IMR90_GM_20220330/name.order.HCG_methy.with_color.DNA_RNA_id.txt",sep="\t",header=F)
m_colors<-mc[,3]
names(m_colors)<-mc[,2]

scale=1000
step=20 
bin_size_align=1 
axistep=1000

files<-list.files(path = ".", pattern = "FOS.IMR90.distal.*.alignedTo.FOS.1200.1.txt")
mf<-NULL
sf_names<-NULL
for(f in files){
	gch<-read.table(f,sep="\t",header=F)
	n<-gsub("FOS.IMR90.distal.", "", f)
	n<-gsub(".sc\\w+.alignedTo.FOS.1200.1.txt", "", n)
	print(n)
	axisSeq<-seq(0-scale, scale, by=step)
	dataSeq<-seq(5+((length(gch[1,])-5)/2)-as.integer(scale/bin_size_align), 5+((length(gch[1,])-5)/2)+as.integer(scale/bin_size_align), by=as.integer(step/bin_size_align))
	valueGch<-NULL
	for(i in dataSeq){
                if(floor(i+step/(2*bin_size_align)-1)-ceiling(i-step/(2*bin_size_align)) <= 0){
                        valueGch<-cbind(valueGch,mean(gch[,ceiling(i-step/(2*bin_size_align))], na.rm=T))
                }else{
                        valueGch<-cbind(valueGch,mean(colMeans(gch[,ceiling(i-step/(2*bin_size_align)):floor(i+step/(2*bin_size_align)-1)], na.rm=T), na.rm=T))
                }

    }
    valueGch[is.na(valueGch)]=mean(valueGch,na.rm=T)
    mf<-rbind(mf,valueGch)
    sf_names<-c(sf_names,n)
}
rownames(mf)<-sf_names


set.seed(42) # Sets seed for reproducibility
for(j in c(5,10,15,20)){
	pdf(paste("FOS_GCH_acc_zscore_1kb.plexity_",j,".pdf",sep=""),paper="special", height=5, width=5)
	tsne_out <- Rtsne(as.matrix(mf),perplexity = j, max_iter=5000) # Run TSNE
	plot(tsne_out$Y,col=m_colors[sf_names],pch=16) # Plot the result
	dev.off()
}

pdf("FOS_GCH_acc_zscore_1kb.umap.pdf",paper="special", height=5, width=5)
umap_out <- umap(as.matrix(mf)) # Run umap
plot(umap_out$layout,col=m_colors[sf_names],pch=16) # Plot the result
dev.off()


#try +/-500bp
m1<-mf[,25:75]
rownames(m1)<-sf_names

set.seed(42) # Sets seed for reproducibility
for(j in c(5,10,15,20)){
	pdf(paste("FOS_GCH_acc_zscore_500bp.plexity_",j,".pdf",sep=""),paper="special", height=5, width=5)
	tsne_out <- Rtsne(as.matrix(m1),perplexity = j, max_iter=5000) # Run TSNE
	plot(tsne_out$Y,col=m_colors[sf_names],pch=16) # Plot the result
	dev.off()
}

pdf("FOS_GCH_acc_zscore_500bp.umap.pdf",paper="special", height=5, width=5)
umap_out <- umap(as.matrix(m1)) # Run umap
plot(umap_out$layout,col=m_colors[sf_names],pch=16) # Plot the result
dev.off()

##FOS 500bp by t-SNE looks better (choose plexity 20)

for(j in seq(0,41)){
	set.seed(j)
	pdf(paste("FOS_GCH_acc_zscore_500bp.plexity_20.seed_",j,".pdf",sep=""),paper="special", height=5, width=5)
	tsne_out <- Rtsne(as.matrix(m1),perplexity = 20, max_iter=5000) # Run TSNE
	plot(tsne_out$Y,col=m_colors[s_names],pch=16) # Plot the result
	dev.off()
}